In [4]:
!pip install kagglehub
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install scikit-learn

   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------- ----------------------------- 2.1/8.1 MB 13.0 MB/s eta 0:00:01
   ----------------------- ---------------- 4.7/8.1 MB 15.0 MB/s eta 0:00:01
   -------------------------------- ------- 6.6/8.1 MB 13.4 MB/s eta 0:00:01
   ---------------------------------------- 8.1/8.1 MB 12.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 15.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ------------------ --------------------- 3.1/7.0 MB 15.3 MB/s eta 0:00:01
   ------------------------------------- -- 6.6/7.0 MB 15.5 MB/s eta 0:00:01
   ---------------------------------------- 7.0/7.0 MB 14.3 MB/s eta 0:00:00

   ---------------------------------------- 0/7 [pyparsing]
   ----- ---------------------------------- 1/7 [pillow]
   ----- ---------------------------------- 1/7 [pillow]
  

In [5]:
import kagglehub
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from kagglehub import KaggleDatasetAdapter

In [8]:
train = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "kaushiksuresh147/customer-segmentation",
  "Train.csv",)

test = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "kaushiksuresh147/customer-segmentation",
  "Test.csv",)

C:\Users\rafid\AppData\Local\Temp\ipykernel_10496\1088270299.py:1: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  train = kagglehub.load_dataset(
C:\Users\rafid\AppData\Local\Temp\ipykernel_10496\1088270299.py:6: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  test = kagglehub.load_dataset(


In [10]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2627 entries, 0 to 2626
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               2627 non-n